<a href="https://colab.research.google.com/github/manuel-suarez/denoising-tests/blob/main/SAR_Image_Despeckling_Using_a_Convolutional_Neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [SAR Image Despeckling Using a Convolutional Neural](https://github.com/TheKiteFlier/SAR-Image-Despeckling-Using-CNN-/blob/main/SAR%20Image%20Despeckling%20Using%20a%20Convolutional.ipynb)

## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Input,Conv2D,ReLU,BatchNormalization,LeakyReLU
import os
from skimage.util import random_noise
import sys
import time
from tqdm.notebook import tqdm
import shutil

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Extramos las imagenes de manera local en el entorno
from zipfile import ZipFile

gdrive_dir = '/content/gdrive'
with ZipFile(os.path.join(gdrive_dir,'MyDrive/DeepLearning/Sentinel 1&2','v_2.zip'), 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall(path='kaggle')

In [ ]:
INPUT_SIZE= (64,64)
BS=16
ROOT_DIR="/kaggle/"

## Preparing data

In [ ]:
DATASET=os.path.join(ROOT_DIR,'input/sentinel12-image-pairs-segregated-by-terrain/v_2')
DATA_GEN_INPUT=os.path.join(ROOT_DIR,'DATASET')

if os.path.exists(DATA_GEN_INPUT):
    shutil.rmtree(DATA_GEN_INPUT)
os.mkdir(DATA_GEN_INPUT)

src=os.path.join(DATASET,"agri/s2")
dst=os.path.join(DATA_GEN_INPUT,"DATA")
os.symlink(src,dst)